## Setup

If running for the first time in a session, execute the next cell and restart your kernel to install 🛰️PACESHIP.

In [ ]:
%pip install https://github.com/pacehackweek/paceship

In [ ]:
import earthaccess
import paceship
import xarray as xr
from cartopy.crs import Orthographic, PlateCarree
from matplotlib import pyplot

## Access Data

In [ ]:
tspan = ("2024-08", "2024-08")
results = earthaccess.search_data(
    short_name="PACE_OCI_L3M_SFREFL",
    granule_name="*.MO.*.0p1deg.*",
    temporal=tspan,
)

In [ ]:
paths = earthaccess.download(results, "granules")

In [ ]:
dataset = xr.open_dataset(paths[0])

In [ ]:
dataset

## Generate Figure

In [ ]:
rgb = dataset["rhos"].sel({"wavelength": [645, 555, 440]}, method="nearest")

In [ ]:
rgb = paceship.l3enhance(rgb)

In [ ]:
plot = rgb.plot.imshow()

## Save as PNG

In [ ]:
fig, ax = pyplot.subplots(figsize=(4, 4), subplot_kw={"projection": Orthographic(-30)})
plot = rgb.plot.imshow(ax=ax, transform=PlateCarree())
pyplot.show()

In [ ]:
fig.savefig(f"{tspan[0]}-blue-marble.png", dpi=72, transparent=True)